The log-odds ratio with an informative (and uninformative) Dirichlet prior (described in [Monroe et al. 2009, Fighting Words](http://languagelog.ldc.upenn.edu/myl/Monroe.pdf)) is a common method for finding distinctive terms in two datasets (see [Jurafsky et al. 2014](https://firstmonday.org/ojs/index.php/fm/article/view/4944/3863) for an example article that uses it to make an empirical argument). This method for finding distinguishing words combines a number of desirable properties:

* it specifies an intuitive metric (the log-odds) for comparing word probabilities in two corpora
* it incorporates prior information in the form of pseudocounts, which can either act as a smoothing factor (in the uninformative case) or incorporate real information about the expected frequency of words overall.
* it accounts for variability of a frequency estimate by essentially converting the log-odds to a z-score.

In this homework you will implement both of these ratios and compare the results.

In [ ]:
import sys, operator, math, nltk
from collections import Counter

In [ ]:
def read_and_tokenize(filename):
    
    with open(filename, encoding="utf-8") as file:
        tokens=[]
        # lowercase
        for line in file:
            data=line.rstrip().lower()
            # This dataset is already tokenized, so we can split on whitespace
            tokens.extend(data.split(" "))
        return tokens

The data we'll use in this case comes from a sample of 1000 positive and 1000 negative movie reviews from the [Large Movie Review Dataset](https://ai.stanford.edu/~amaas/data/sentiment/).  The version of the data used in this homework has already been tokenized for you.

In [ ]:
negative_tokens=read_and_tokenize("../data/negative.reviews.txt")
positive_tokens=read_and_tokenize("../data/positive.reviews.txt")

Q1.  Implement the log-odds ratio with an uninformative Dirichlet prior.  This value, $\hat\zeta_w^{(i-j)}$ for word $w$ reflecting the difference in usage between corpus $i$ and corpus $j$, is given by the following equation:

$$
\hat\zeta_w^{(i-j)}= {\hat{d}_w^{(i-j)} \over \sqrt{\sigma^2\left(\hat{d}_w^{(i-j)}\right)}}
$$

Where: 

$$
\hat{d}_w^{(i-j)} = \log \left({y_w^i + \alpha_w} \over {n^i + \alpha_0 - y_w^i - \alpha_w}) \right) -  \log \left({y_w^j + \alpha_w} \over {n^j + \alpha_0 - y_w^j - \alpha_w}) \right)
$$

$$
\sigma^2\left(\hat{d}_w^{(i-j)}\right) \approx {1 \over {y_w^i + \alpha_w}} + {1 \over {y_w^j + \alpha_w} }
$$

And:

* $y_w^i = $ count of word $w$ in corpus $i$ (likewise for $j$)
* $\alpha_w$ = 0.01
* $V$ = size of vocabulary (number of distinct word types)
* $\alpha_0 = V * \alpha_w$
* $n^i = $ number of words in corpus $i$ (likewise for $j$)

Here the two corpora are the positive movie reviews (e.g., $i$ = positive) and the negative movie reviews (e.g., $j$ = negative). Using this metric, print out the 25 words most strongly aligned with the positive corpus, and 25 words most strongly aligned with the negative corpus.

In [ ]:
def logodds_with_uninformative_prior(one_tokens, two_tokens, display=25):
    
    # complete this section

In [ ]:
logodds_with_uninformative_prior(positive_tokens, negative_tokens)

Q2: As you increase the constant $\alpha_w$ in the equations above, what happens to $\hat\zeta_w^{(i-j)}$, $\hat{d}_w^{(i-j)}$ and $\sigma^2\left(\hat{d}_w^{(i-j)}\right)$ (i.e., do they get bigger or smaller)?  Answer this by plugging the following values in your implementation of these two quantities, and varying $\alpha_w$ (and, consequently, $\alpha_0$).

* $y_w^i=34$
* $y_w^j=17$
* $n^i=1000$
* $n^j=1000$
* $V=500$

Now let's make that prior informative by including information about the overall frequency of a given word in a background corpus (i.e., a corpus that represents general word usage, without regard for labeled subcorpora).  To do so, there are only two small changes to make:

* We need to gather a background corpus $b$ and calculate $\hat\pi_w$, the relative frequency of word $w$ in $b$ (i.e., the number of times $w$ occurs in $b$ divided by the number of words in $b$).

* In the uninformative prior above, $\alpha_w$ was a constant (0.01) and $\alpha_0 = V * \alpha_w$.  Let us now set $\alpha_0 = 1000$ and $\alpha_w = \hat\pi_w * \alpha_0$.  This reflects a pseudocount capturing the fractional number of times we would expect to see word $w$ in a sample of 1000 words.

This allows us to specify that a common word like "the" (which has a relative frequency of $\approx 0.04$) would have $\alpha_w = 40$, while an infrequent word like "beneficiaries" (relative frequency $\approx 0.00002$) would have $\alpha_w = 0.02$. 

Q3. Implement a log-odds ratio with informative prior, using a larger background corpus of 5M tokens drawn from the same dataset (given to you as `priors` below, which contains the relative frequencies of words calculated from that corpus) and set $\alpha_0 = 1000$. Using this metric, print out again the 25 words most strongly aligned with the positive corpus, and 25 words most strongly aligned with the negative corpus.  Is there a meaningful difference?

In [ ]:
def read_priors(filename):
    counts=Counter()
    freqs={}
    tokens=read_and_tokenize(filename)
    total=len(tokens)

    for token in tokens:
        counts[token]+=1

    for word in counts:
        freqs[word]=counts[word]/total

    return freqs
    
priors=read_priors("../data/sentiment.background.txt")

In [ ]:
def logodds_with_informative_prior(one_tokens, two_tokens, priors, display=25):
    
    # complete this section

In [ ]:
logodds_with_informative_prior(positive_tokens, negative_tokens, priors)